I am sharing this note bbok to illustrate
- how to convert token probability to word prediction (token to word aligment) for ensemble, I use different tokenizers for the two models.
- results:

fold0 
- bigbird-large    : CV 0.652873
- longformer-large : CV 0.672632
- bigbird-large  + longformer-large : CV 0.683076 (LB 0.682)

fold1 
- bigbird-large    : CV 0.676453
- longformer-large : CV 0.681581
- bigbird-large  + longformer-large : CV 0.696001 (LB 0.681)


fold0+fold1 : LB 0.690

In [ ]:
import sys
sys.path.append('../input/feedback-prize-submit-01')

import numpy as np
import glob
import pandas as pd
from timeit import default_timer as timer

import torch
from torch.nn.parallel.data_parallel import data_parallel

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import *


import torch.cuda.amp as amp
is_amp   = True  #True #False
is_cuda  = True
is_debug = False


#helper
def time_to_str(t, mode='min'):
    if mode=='min':
        t  = int(t)/60
        hr = t//60
        min = t%60
        return '%2d hr %02d min'%(hr,min)
    elif mode=='sec':
        t   = int(t)
        min = t//60
        sec = t%60
        return '%2d min %02d sec'%(min,sec)
    else:
        raise NotImplementedError

In [ ]:
#config 

discourse_marker_to_label = {
    'O': 0,
    'B-Lead': 1,
    'I-Lead': 2,
    'B-Position': 3,
    'I-Position': 4,
    'B-Claim': 5,
    'I-Claim': 6,
    'B-Counterclaim': 7,
    'I-Counterclaim': 8,
    'B-Rebuttal': 9,
    'I-Rebuttal': 10,
    'B-Evidence': 11,
    'I-Evidence': 12,
    'B-Concluding Statement': 13,
    'I-Concluding Statement': 14,
    'IGNORE': -100,
}
label_to_discourse_marker = {v: k for k, v in discourse_marker_to_label.items()}
num_discourse_marker = 15 #len(label_to_discourse_marker)-1 #15

length_threshold = {
    'Lead'                : 9,
    'Position'            : 5,
    'Claim'               : 3,
    'Counterclaim'        : 6,
    'Rebuttal'            : 4,
    'Evidence'            : 14,
    'Concluding Statement': 11,
}
probability_threshold = {
    'Lead'                : 0.70,
    'Position'            : 0.55,
    'Claim'               : 0.55,
    'Counterclaim'        : 0.50,
    'Rebuttal'            : 0.55,
    'Evidence'            : 0.65,
    'Concluding Statement': 0.70,
}
max_length = 1600

#-------------------------------
submit_dir = ''

if is_debug:
    text_dir = '../input/feedback-prize-2021/train'
    df = pd.read_csv('../input/feedback-prize-submit-01/train.fold10.csv')
    valid_df = df[df['fold'] == 0].reset_index(drop=True)
    valid_id = valid_df['id'].unique()

else:
    text_dir = '../input/feedback-prize-2021/test'
    valid_id = [ f.split('/')[-1][:-4] for f in glob.glob(text_dir+'/*.txt') ]
    valid_id = sorted(valid_id)
num_valid = len(valid_id)
print('len(valid_id)',len(valid_id))

# print(valid_id)



In [ ]:
#data

df_text=[]
for id in valid_id:
    text_file = text_dir +'/%s.txt'%id
    with open(text_file, 'r') as f:
        text = f.read()

    text = text.replace(u'\xa0', u' ')
    text = text.rstrip()
    text = text.lstrip()
    df_text.append((id,text))
df_text = pd.DataFrame(df_text, columns=['id','text'])
print('df_text.shape',df_text.shape)
print(df_text)

class FeedbackDataset(Dataset):
    def __init__(self, df_text, tokenizer, max_length = 1600):

        self.df_text  = df_text
        self.max_length = max_length
        self.tokenizer = tokenizer
        self.length = len(self.df_text)

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        d    = self.df_text.iloc[index]
        id   = d['id']
        text = d.text

        #text to token
        encoded = self.tokenizer.encode_plus(
            text,
            add_special_tokens=False,
            return_offsets_mapping=True,
            max_length=4096, #<todo>
            truncation=True,
        )
        token_id     =  encoded['input_ids']
        token_offset =  encoded['offset_mapping']

        # add end, start token id
        token_id = [self.tokenizer.cls_token_id] + token_id
        token_id = token_id[: self.max_length - 1]
        token_id = token_id + [self.tokenizer.sep_token_id]

        # padding
        token_mask = [1] * len(token_id)

        padding_length = max_length - len(token_id)
        if padding_length > 0:
            if self.tokenizer.padding_side == 'right':
                token_id    = token_id    + [self.tokenizer.pad_token_id] * padding_length
                token_mask  = token_mask  + [0] * padding_length
            else:
                raise NotImplementedError

        #-------------------------------------
        r = {}
        r['index'] = index
        r['id'   ] = id
        r['text' ] = text
        r['token_offset'] = str(token_offset) #force batch loader store as list
        r['token_id'    ] = torch.tensor(token_id,    dtype=torch.long)
        r['token_mask'  ] = torch.tensor(token_mask,  dtype=torch.long)
        return r



In [ ]:
#net

from bigbird_base_model import Net as BidBirdBaseNet
from longformer_base_model import Net as LongformerBaseNet
from bigbird_large_model import Net as BidBirdLargeNet
from longformer_large_model import Net as LongformerLargeNet


checkpoint =[
#     '../input/feedback-prize-submit-01/bigbird-base-10kf-1600-03-fold-0-00018000.model.pth',
#     '../input/feedback-prize-submit-01/bigbird-base-10kf-1600-03-fold-1-00021000.model.pth',
#     '../input/feedback-prize-submit-01/longformer-base-10kf-1600-03-fold-0-00021000.model.pth',
#     '../input/feedback-prize-submit-01/longformer-base-10kf-1600-03-fold-1-00018000.model.pth',
    '../input/feedback-prize-submit-01/bigbird-large-10kf-1600-03-fold-0-00015000.model.pth',  #4hr for 6 model
    '../input/feedback-prize-submit-01/longformer-large-10kf-1600-03-fold-0-00018000.model.pth',
]

net_type = [
    #BidBirdBaseNet(arch='google/bigbird-roberta-base'),
    #LongformerBaseNet(arch='allenai/longformer-base-4096'),
    
#     [BidBirdBaseNet,   '../input/feedback-prize-submit-01/google-bigbird-roberta-base' ],
#     [BidBirdBaseNet,   '../input/feedback-prize-submit-01/google-bigbird-roberta-base' ],
#     [LongformerBaseNet,'../input/feedback-prize-submit-01/allenai-longformer-base-4096'],
#     [LongformerBaseNet,'../input/feedback-prize-submit-01/allenai-longformer-base-4096'],
    [BidBirdLargeNet,   '../input/feedback-prize-submit-01/google-bigbird-roberta-large' ],
    [LongformerLargeNet,'../input/feedback-prize-submit-01/allenai-longformer-large-4096'],
]
num_net=len(net_type)


In [ ]:
#processing

def text_to_word(text):
    word = text.split()
    word_offset = []

    start = 0
    for w in word:
        r = text[start:].find(w)

        if r==-1:
            raise NotImplementedError
        else:
            start = start+r
            end   = start+len(w)
            word_offset.append((start,end))
            #print('%32s'%w, '%5d'%start, '%5d'%r, text[start:end])
        start = end

    return word, word_offset

def word_probability_to_predict_df(text_to_word_probability, id):
    len_word = len(text_to_word_probability)
    word_predict = text_to_word_probability.argmax(-1)
    word_score   = text_to_word_probability.max(-1)
    predict_df = []

    t = 0
    while 1:
        if word_predict[t] not in [
            discourse_marker_to_label['O'],
            discourse_marker_to_label['IGNORE'],
        ]:
            start = t
            b_marker_label = word_predict[t]
        else:
            t = t+1
            if t== len_word-1: break
            continue

        t = t+1
        if t== len_word-1: break

        #----
        if   label_to_discourse_marker[b_marker_label][0]=='B':
            i_marker_label = b_marker_label+1
        elif label_to_discourse_marker[b_marker_label][0]=='I':
            i_marker_label = b_marker_label
        else:
            raise NotImplementedError

        while 1:
            #print(t)
            if (word_predict[t] != i_marker_label) or (t ==len_word-1):
                end = t
                prediction_string = ' '.join([str(i) for i in range(start,end)]) #np.arange(start,end).tolist()
                discourse_type = label_to_discourse_marker[b_marker_label][2:]
                discourse_score = word_score[start:end].tolist()
                predict_df.append((id, discourse_type, prediction_string, str(discourse_score)))
                #print(predict_df[-1])
                break
            else:
                t = t+1
                continue
        if t== len_word-1: break

    predict_df = pd.DataFrame(predict_df, columns=['id', 'class', 'predictionstring', 'score'])
    return predict_df

def do_threshold(submit_df, use=['length','probability']):
    df = submit_df.copy()
    df = df.fillna('')

    if 'length' in use:
        df['l'] = df.predictionstring.apply(lambda x: len(x.split()))
        for key, value in length_threshold.items():
            #value=3
            index = df.loc[df['class'] == key].query('l<%d'%value).index
            df.drop(index, inplace=True)

    if 'probability' in use:
        df['s'] = df.score.apply(lambda x: np.mean(eval(x)))
        for key, value in probability_threshold.items():
            index = df.loc[df['class'] == key].query('s<%f'%value).index
            df.drop(index, inplace=True)

    df = df[['id', 'class', 'predictionstring']]
    return df

#evaluation for debug ----
# https://www.kaggle.com/cpmpml/faster-metric-computation

def compute_overlap(predict, truth):
    """
    Calculates the overlap between prediction and
    ground truth and overlap percentages used for determining
    true positives.
    """
    # Length of each and intersection
    try:
        len_truth   = len(truth)
        len_predict = len(predict)
        intersect = len(truth & predict)
        overlap1 = intersect/ len_truth
        overlap2 = intersect/ len_predict
        return (overlap1, overlap2)
    except:  # at least one of the input is NaN
        return (0, 0)

def compute_f1_score_one(predict_df, truth_df, discourse_type):
    """
    A function that scores for the kaggle
        Student Writing Competition

    Uses the steps in the evaluation page here:
        https://www.kaggle.com/c/feedback-prize-2021/overview/evaluation
    """
    t_df = truth_df.loc[truth_df['discourse_type'] == discourse_type,   ['id', 'predictionstring']].reset_index(drop=True)
    p_df = predict_df.loc[predict_df['class'] == discourse_type,  ['id', 'predictionstring']].reset_index(drop=True)

    p_df.loc[:,'predict_id'] = p_df.index
    t_df.loc[:,'truth_id'] = t_df.index
    p_df.loc[:,'predictionstring'] = [set(p.split(' ')) for p in p_df['predictionstring']]
    t_df.loc[:,'predictionstring'] = [set(p.split(' ')) for p in t_df['predictionstring']]

    # Step 1. all ground truths and predictions for a given class are compared.
    joined = p_df.merge(t_df,
                           left_on='id',
                           right_on='id',
                           how='outer',
                           suffixes=('_p','_t')
                          )
    overlap = [compute_overlap(*predictionstring) for predictionstring in zip(joined.predictionstring_p, joined.predictionstring_t)]

    # 2. If the overlap between the ground truth and prediction is >= 0.5,
    # and the overlap between the prediction and the ground truth >= 0.5,
    # the prediction is a match and considered a true positive.
    # If multiple matches exist, the match with the highest pair of overlaps is taken.
    joined['potential_TP'] = [(o[0] >= 0.5 and o[1] >= 0.5) for o in overlap]
    joined['max_overlap' ] = [max(*o) for o in overlap]
    joined_tp = joined.query('potential_TP').reset_index(drop=True)
    tp_pred_ids = joined_tp\
        .sort_values('max_overlap', ascending=False) \
        .groupby(['id','truth_id'])['predict_id'].first()

    # 3. Any unmatched ground truths are false negatives
    # and any unmatched predictions are false positives.
    fp_pred_ids = set(joined['predict_id'].unique()) - set(tp_pred_ids)

    matched_gt_ids   = joined_tp['truth_id'].unique()
    unmatched_gt_ids = set(joined['truth_id'].unique()) -  set(matched_gt_ids)

    # Get numbers of each type
    TP = len(tp_pred_ids)
    FP = len(fp_pred_ids)
    FN = len(unmatched_gt_ids)
    f1 = TP / (TP + 0.5*(FP+FN))
    return f1

def compute_lb_f1_score(predict_df, truth_df):
    f1_score = {}
    for discourse_type in truth_df.discourse_type.unique():
        f1_score[discourse_type] = compute_f1_score_one(predict_df, truth_df, discourse_type)
    #f1 = np.mean([v for v in class_scores.values()])
    return f1_score



In [ ]:
## main submission function !!!!
def run_submit():
    if is_debug: print("THIS IS DEBUG ####################################")

    results = []
    for n in range(num_net):
        Net, arch = net_type[n]
        net = Net(arch)


        net.load_state_dict(torch.load(checkpoint[n], map_location=lambda storage, loc: storage)['state_dict'],strict=False)
        tokenizer = net.get_tokenizer()
        if is_cuda:
            net.cuda()
        print('load ok : [%d] %s'%(n,net.arch))
        print('              %s'%(checkpoint[n]))

        valid_dataset = FeedbackDataset(df_text, tokenizer, max_length)
        valid_loader  = DataLoader(
            valid_dataset,
            sampler = SequentialSampler(valid_dataset),
            batch_size  = 8, #4, #
            drop_last   = False,
            num_workers = 2, #0, #
            pin_memory  = False,
            #collate_fn = null_collate_fn,
        )

        # start here !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
        results_n = {
            'id':[],
            'token_mask':[],
            'token_offset':[],
            'probability':[],
        }

        T = 0
        start_timer = timer()
        for t, batch in enumerate(valid_loader):
            batch_size = len(batch['index'])
            token_mask = batch['token_mask']
            token_id   = batch['token_id']
            if is_cuda:
                token_mask = token_mask.cuda()
                token_id = token_id.cuda()

            net.eval()
            with torch.no_grad():
                with amp.autocast(enabled=is_amp):
                    probability = data_parallel(net,(token_id, token_mask))
                    #probability = net[n](token_id, token_mask)

                    results_n['probability'].append( (probability*255).byte().data.cpu().numpy() )
                    results_n['token_offset' ] += [eval(x) for x in batch['token_offset']]
                    results_n['token_mask' ].append( token_mask.bool().data.cpu().numpy())
                    results_n['id' ] += batch['id']
                    T += batch_size

            print('\r\t%d/%d  %s'%(T, len(valid_dataset), time_to_str(timer() - start_timer,'sec')),end='',flush=True)

        #----------------------------
        if is_cuda: torch.cuda.empty_cache()
        print('')

        results.append({
            'probability' : np.concatenate(results_n['probability']),
            'token_offset': np.array(results_n['token_offset'], object)
            #'token_mask'] = np.concatenate(results['token_mask'])
            #'id'] = np.array(results['id' ])
        })
    #------------------------------------------------------------------------

    submit_df = []
    for i in range(num_valid):
        d  = df_text.iloc[i]
        id = d.id
        text = d.text
        word, word_offset = text_to_word(text)
        #print(i,id[i], len(text), len(word))

        #ensemble -----
        token_to_text_probability = np.full((len(text),num_discourse_marker),0, np.float32)
        for j in range(num_net):
            p = results[j]['probability'][i][1:]/255
            for t,(start,end) in enumerate(results[j]['token_offset'][i]):
                if t==max_length-1: break #assume max_length, else use token_mask to get length
                token_to_text_probability[start:end]+=p[t] #**0.5
        token_to_text_probability = token_to_text_probability/num_net
        #ensemble -----


        text_to_word_probability = np.full((len(word),num_discourse_marker),0, np.float32)
        for t,(start,end) in enumerate(word_offset):
            text_to_word_probability[t]=token_to_text_probability[start:end].mean(0)

        predict_df = word_probability_to_predict_df(text_to_word_probability, id)
        submit_df.append(predict_df)
        #print('\r preparing submit_df :', i, id, len(text), len(word), end ='', flush=True)
    print('')

    #----------------------------------------
    submit_df = pd.concat(submit_df).reset_index(drop=True) 
    submit_df = do_threshold(submit_df, use=['length', 'probability'])
    submit_df.to_csv('submission.csv', index=False)

    print('----')
    for t in range(3): print(submit_df.iloc[t],'\n')
    print('submission ok!----')
    if is_debug:
        f1_score = compute_lb_f1_score(submit_df, valid_df)
        print('f1 macro : %f\n' % np.mean([v for v in f1_score.values()]))
        for k,v in f1_score.items():
            print('%20s : %05f'%(k,v))


In [ ]:
#check function
def run_check_dataset():

    tokenizer = net[0].get_tokenizer()
    dataset = FeedbackDataset(df_text, tokenizer, max_length)

    for i in range(5):
        r = dataset[i]
        print(r['index'],'-----------')
        for k in ['token_id', 'token_mask']:
            v = r[k]
            print(k)
            print('\t',v.shape, v.is_contiguous())
            print('\t',v)
        print('')

In [ ]:
#run_check_dataset()
run_submit()